<a href="https://colab.research.google.com/github/wontaaail/S.T.Geometry/blob/master/20191128*1206%20(5).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using Dropout on the Visible Layer

In [13]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.constraints import maxnorm
from keras.optimizers import SGD
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [0]:
seed = 6

In [0]:
dataframe = pandas.read_csv("/content/gdrive/My Drive/Colab Notebooks/code/sonar.csv", header=None)
dataset = dataframe.values

In [0]:
X = dataset[:,0:60].astype(float)
Y = dataset[:,60]

In [0]:
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)

In [0]:
def create_model():
  model = Sequential()
  model.add(Dense(60, input_dim=60, kernel_initializer= 'normal' , activation= 'relu' , kernel_constraint=maxnorm(3)))
  model.add(Dropout(0.2))
  model.add(Dense(30, kernel_initializer= 'normal' , activation= 'relu' , kernel_constraint=maxnorm(3)))
  model.add(Dropout(0.2))
  model.add(Dense(1, kernel_initializer= 'normal' , activation= 'sigmoid' ))
  sgd = SGD(lr=0.01, momentum=0.9, decay=0.0, nesterov=False)
  model.compile(loss= 'binary_crossentropy' , optimizer=sgd, metrics=[ 'accuracy' ])
  return model

In [0]:
numpy.random.seed(seed)

In [0]:
estimators = []
estimators.append(( 'standardize' , StandardScaler()))
estimators.append(( 'mlp' , KerasClassifier(build_fn=create_model, epochs=300, batch_size=16, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

In [0]:
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)

In [19]:
print("Hidden: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Hidden: 85.15% (6.39%)
